# Data cleaning of a csv file containing product infos

INFMDI721 - TP - 23/10/2019

Consigne :
Dans ce dataset: https://raw.githubusercontent.com/fspot/INFMDI-721/master/lesson5/products.csv, chaque ligne correspond à un produit alimentaire mis en vente par un utilisateur.

- On aimerait avoir une colonne de prix unifiés en euros. Problème: la currency n'est pas indiquée pour tous les produits: il va falloir essayer de "deviner" les currency manquantes, en se basant sur l'adresse IP de l'utilisateur.
- La colonne "infos" liste des ingrédients présents dans le produit. On préfèrerait avoir une colonne de type bool par ingrédient, indiquant si le produit contient ou non cet ingrédient.

Voici une liste d'APIs qui peut vous être utile : https://github.com/public-apis/public-apis (mais vous pouvez en utiliser d'autres si vous le voulez).

In [1]:
import pandas as pd
import json
import requests
import re

# Récupération du csv

In [2]:
df = pd.read_csv("products.csv", sep=";")
df.head(10)

,username,ip_address,product,price,infos
0,ldrover0,666.666.666.666,Clam - Cherrystone,712.8,May contain sugar
1,kizakov1,nope,Soup - Campbells Bean Medley,379.26,Contains peanut and fish
2,abromet2,240.177.79.234,Island Oasis - Lemonade,305.96,Ingredients: mustard and fish
3,kkarolowski3,26.191.237.49,"Water - Mineral, Natural",350.15,Contains gluten
4,mbuckney4,58.90.204.239,Radish - Pickled,949.79,"May contain sugar, egg and fish"
5,bsnozzwell5,226.52.32.70,Oil - Sesame,354.33 MGA,Ingredients: sugar and milk
6,afairholme6,127.197.204.119,Chicken - Tenderloin,484.83,May contain sugar
7,avowdon7,189.169.17.54,Dc Hikiage Hira Huba,111.56,Contains sugar
8,epridham8,187.129.113.105,Dried Figs,88.05,"Ingredients: sugar, milk and fish"
9,tkendrew9,22.32.234.215,Pop - Club Soda Can,861.25,"May contain peanut, sugar, milk and fish"


# Transformation du prix

## Récupération du pays à partir de l'IP

In [3]:
with open("credentials.json") as f:
    data = json.load(f)
    token = data['token_ipgeolocation']

In [4]:
# Connexion à l'API ipgeolocation pour récupérer le pays d'origine d'une IP
def get_country(ip):
    url = "https://api.ipgeolocation.io/ipgeo?apiKey="+token+"&ip="+ip
    response = requests.get(url)
    content = response.json()
    if "country_code2" in content:
        return content["country_code2"]
    return ""

In [5]:
df["country"] = df["ip_address"].apply(lambda x: get_country(x))

In [6]:
df.head()

,username,ip_address,product,price,infos,country
0,ldrover0,666.666.666.666,Clam - Cherrystone,712.8,May contain sugar,
1,kizakov1,nope,Soup - Campbells Bean Medley,379.26,Contains peanut and fish,
2,abromet2,240.177.79.234,Island Oasis - Lemonade,305.96,Ingredients: mustard and fish,
3,kkarolowski3,26.191.237.49,"Water - Mineral, Natural",350.15,Contains gluten,US
4,mbuckney4,58.90.204.239,Radish - Pickled,949.79,"May contain sugar, egg and fish",JP


## Récupération de la monnaie

In [7]:
# Chargement d'un dictionnaire faisant la relation country -> currency
with open("countries_currencies.json") as cc:  # json source : http://country.io
    countries_currencies = json.load(cc)
countries_currencies[""] = ""

In [8]:
def get_currency(row):
    res = re.findall(r"(\w{3})$", row["price"])
    if len(res) > 0:
        return res[0]
    else:
        if (row["country"] in countries_currencies):
            return countries_currencies[row["country"]]
    return ''

In [9]:
#df["currency"] = df["country"].map(countries_currencies)
df["currency"] = df.apply(get_currency, axis=1)

In [10]:
df.head(10)

,username,ip_address,product,price,infos,country,currency
0,ldrover0,666.666.666.666,Clam - Cherrystone,712.8,May contain sugar,,
1,kizakov1,nope,Soup - Campbells Bean Medley,379.26,Contains peanut and fish,,
2,abromet2,240.177.79.234,Island Oasis - Lemonade,305.96,Ingredients: mustard and fish,,
3,kkarolowski3,26.191.237.49,"Water - Mineral, Natural",350.15,Contains gluten,US,USD
4,mbuckney4,58.90.204.239,Radish - Pickled,949.79,"May contain sugar, egg and fish",JP,JPY
5,bsnozzwell5,226.52.32.70,Oil - Sesame,354.33 MGA,Ingredients: sugar and milk,,MGA
6,afairholme6,127.197.204.119,Chicken - Tenderloin,484.83,May contain sugar,,
7,avowdon7,189.169.17.54,Dc Hikiage Hira Huba,111.56,Contains sugar,MX,MXN
8,epridham8,187.129.113.105,Dried Figs,88.05,"Ingredients: sugar, milk and fish",MX,MXN
9,tkendrew9,22.32.234.215,Pop - Club Soda Can,861.25,"May contain peanut, sugar, milk and fish",US,USD


## Récupération des taux de change actuels via une API

In [11]:
# Requêtage de l'API
url = "https://api.exchangeratesapi.io/latest"
response = requests.get(url)
content = response.json()
exchange_rates = {}
if "rates" in content:
    exchange_rates = content["rates"]
exchange_rates

{'AUD': 1.6237,
 'BGN': 1.9558,
 'BRL': 4.59,
 'CAD': 1.4581,
 'CHF': 1.1004,
 'CNY': 7.8795,
 'CZK': 25.57,
 'DKK': 7.4708,
 'GBP': 0.86065,
 'HKD': 8.7295,
 'HRK': 7.4405,
 'HUF': 329.92,
 'IDR': 15626.52,
 'ILS': 3.9411,
 'INR': 78.9245,
 'ISK': 139.3,
 'JPY': 120.87,
 'KRW': 1305.83,
 'MXN': 21.2721,
 'MYR': 4.6618,
 'NOK': 10.1833,
 'NZD': 1.7358,
 'PHP': 56.969,
 'PLN': 4.2778,
 'RON': 4.7609,
 'RUB': 70.9699,
 'SEK': 10.7303,
 'SGD': 1.5164,
 'THB': 33.735,
 'TRY': 6.4897,
 'USD': 1.113,
 'ZAR': 16.3337}

## Formatage de la colonne prix

In [12]:
def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False

In [13]:
def clean_price(price):
    if (price == None or len(price) == 0):
        return None
    price = price.strip()
    res = re.findall(r"(-?[0-9]\d*\.\d+)", price)
    if len(res) > 0:
        price = res[0]
    if is_number(price):
        return float(price)
    return None

In [14]:
df["price"] = df["price"].apply(lambda x: clean_price(x))

In [15]:
df.head()

,username,ip_address,product,price,infos,country,currency
0,ldrover0,666.666.666.666,Clam - Cherrystone,712.80,May contain sugar,,
1,kizakov1,nope,Soup - Campbells Bean Medley,379.26,Contains peanut and fish,,
2,abromet2,240.177.79.234,Island Oasis - Lemonade,305.96,Ingredients: mustard and fish,,
3,kkarolowski3,26.191.237.49,"Water - Mineral, Natural",350.15,Contains gluten,US,USD
4,mbuckney4,58.90.204.239,Radish - Pickled,949.79,"May contain sugar, egg and fish",JP,JPY


## Conversion du prix en euro

In [16]:
def convert_price(row):
    if row["price"] == None:
        return None
    if row["currency"] == "EUR":
        return row["currency"]
    if row["currency"] in exchange_rates:
        return round(row["price"]/exchange_rates[row["currency"]],2)
    return None

In [17]:
df["price_eur"] = df.apply(convert_price, axis=1)

In [18]:
df.head(10)

,username,ip_address,product,price,infos,country,currency,price_eur
0,ldrover0,666.666.666.666,Clam - Cherrystone,712.80,May contain sugar,,,None
1,kizakov1,nope,Soup - Campbells Bean Medley,379.26,Contains peanut and fish,,,None
2,abromet2,240.177.79.234,Island Oasis - Lemonade,305.96,Ingredients: mustard and fish,,,None
3,kkarolowski3,26.191.237.49,"Water - Mineral, Natural",350.15,Contains gluten,US,USD,314.6
4,mbuckney4,58.90.204.239,Radish - Pickled,949.79,"May contain sugar, egg and fish",JP,JPY,7.86
5,bsnozzwell5,226.52.32.70,Oil - Sesame,354.33,Ingredients: sugar and milk,,MGA,None
6,afairholme6,127.197.204.119,Chicken - Tenderloin,484.83,May contain sugar,,,None
7,avowdon7,189.169.17.54,Dc Hikiage Hira Huba,111.56,Contains sugar,MX,MXN,5.24
8,epridham8,187.129.113.105,Dried Figs,88.05,"Ingredients: sugar, milk and fish",MX,MXN,4.14
9,tkendrew9,22.32.234.215,Pop - Club Soda Can,861.25,"May contain peanut, sugar, milk and fish",US,USD,773.81


# Obtention des informations sur les ingrédients

## Formatage de la colonne infos

In [19]:
def clean_infos(infos):
    infos = infos.replace("May contain", "").replace("Contains", "").replace("Ingredients:", "").replace(" and ", ", ").strip()
    return infos.split(",")

In [20]:
df["infos_copy"] = df["infos"].apply(lambda x: clean_infos(x))
df.head(10)

,username,ip_address,product,price,infos,country,currency,price_eur,infos_copy
0,ldrover0,666.666.666.666,Clam - Cherrystone,712.80,May contain sugar,,,None,[sugar]
1,kizakov1,nope,Soup - Campbells Bean Medley,379.26,Contains peanut and fish,,,None,"[peanut, fish]"
2,abromet2,240.177.79.234,Island Oasis - Lemonade,305.96,Ingredients: mustard and fish,,,None,"[mustard, fish]"
3,kkarolowski3,26.191.237.49,"Water - Mineral, Natural",350.15,Contains gluten,US,USD,314.6,[gluten]
4,mbuckney4,58.90.204.239,Radish - Pickled,949.79,"May contain sugar, egg and fish",JP,JPY,7.86,"[sugar, egg, fish]"
5,bsnozzwell5,226.52.32.70,Oil - Sesame,354.33,Ingredients: sugar and milk,,MGA,None,"[sugar, milk]"
6,afairholme6,127.197.204.119,Chicken - Tenderloin,484.83,May contain sugar,,,None,[sugar]
7,avowdon7,189.169.17.54,Dc Hikiage Hira Huba,111.56,Contains sugar,MX,MXN,5.24,[sugar]
8,epridham8,187.129.113.105,Dried Figs,88.05,"Ingredients: sugar, milk and fish",MX,MXN,4.14,"[sugar, milk, fish]"
9,tkendrew9,22.32.234.215,Pop - Club Soda Can,861.25,"May contain peanut, sugar, milk and fish",US,USD,773.81,"[peanut, sugar, milk, fish]"


## Création d'une colonne par ingrédient

In [21]:
for index, row in df.iterrows():
    for ingredient in row["infos_copy"]:
        df.at[index, ingredient] = True

In [22]:
df = df.fillna("")
df.head(10)

,username,ip_address,product,price,infos,country,currency,price_eur,infos_copy,sugar,...,gluten,egg,milk,sugar,soja,soja,egg,mustard,fish,
0,ldrover0,666.666.666.666,Clam - Cherrystone,712.80,May contain sugar,,,,[sugar],True,...,,,,,,,,,,
1,kizakov1,nope,Soup - Campbells Bean Medley,379.26,Contains peanut and fish,,,,"[peanut, fish]",,...,,,,,,,,,,
2,abromet2,240.177.79.234,Island Oasis - Lemonade,305.96,Ingredients: mustard and fish,,,,"[mustard, fish]",,...,,,,,,,,,,
3,kkarolowski3,26.191.237.49,"Water - Mineral, Natural",350.15,Contains gluten,US,USD,314.6,[gluten],,...,True,,,,,,,,,
4,mbuckney4,58.90.204.239,Radish - Pickled,949.79,"May contain sugar, egg and fish",JP,JPY,7.86,"[sugar, egg, fish]",True,...,,True,,,,,,,,
5,bsnozzwell5,226.52.32.70,Oil - Sesame,354.33,Ingredients: sugar and milk,,MGA,,"[sugar, milk]",True,...,,,True,,,,,,,
6,afairholme6,127.197.204.119,Chicken - Tenderloin,484.83,May contain sugar,,,,[sugar],True,...,,,,,,,,,,
7,avowdon7,189.169.17.54,Dc Hikiage Hira Huba,111.56,Contains sugar,MX,MXN,5.24,[sugar],True,...,,,,,,,,,,
8,epridham8,187.129.113.105,Dried Figs,88.05,"Ingredients: sugar, milk and fish",MX,MXN,4.14,"[sugar, milk, fish]",True,...,,,True,,,,,,,
9,tkendrew9,22.32.234.215,Pop - Club Soda Can,861.25,"May contain peanut, sugar, milk and fish",US,USD,773.81,"[peanut, sugar, milk, fish]",,...,,,True,True,,,,,,


**Possibilités d'améliorations:**
- Optimiser la création d'une colonne par ingrédient. df.iterrows() est une méthode lente.
- Supprimer les colonnes inutiles, en fonction du besoin.
- Vérifier que les adresses ip sont valides.